In [1]:
import pandas as pd
import numpy as np
import re,requests
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials='MzU0M2RmZmMtZTZiYS00NDY2LWE1ODQtZTUzMTFhNTBjZmQ0OjcxYWE1NGY1LWUxZTctNDk1NC1hZGViLTkwOTIyODNiOWVhYw==', verify_ssl_certs=False,temperature=0.7)

messages = [
    SystemMessage(
        content="Ты лучший аналитик в компании РЖД, необходмо проанализировать данные респондента и вывести вероятность того, уйдет ли респондент от компании и перестанет пользоваться их услугами. Ты даёшь свой ответ только в виде % ухода пользователя и обоснование поставленного процента"
    )
]

In [2]:
def parse_xls(filename: str):
    file = pd.read_excel(filename)
    rows = file.loc
    lst = []
    for row in file.itertuples(index=False):
        rw = []
        for r in row:
            rw.append(r)
        lst.append(rw)
    return lst

In [3]:
waitings = parse_xls("Интересы.xls")
zayavki = parse_xls("Обращения.xls")
obama = parse_xls("Объёмы перевозок.xls")
all_data = [waitings, zayavki, obama]

In [57]:
id = int(input("Введите ID:"))
for i in all_data:
    for k in i:
        for z in k:
            if z == id:
                print(k)
                break
            


['11.04.2024 10:05', 'Привлечение  к новым услугам по перевозке ГЭ (111) Йошкар-Ола-Ревда/Екатеринбург', '6. Продажа услуг действующим клиентам', 'Завершен неудачно', 5087, 'ЦПУ КАЗАНЬ', 2465.0, nan, '11.04.2024 10:30:00', 'Телефонный звонок', '10. Холодный звонок', '0001-094391', 'Интерес "Привлечение  к новым услугам по перевозке ГЭ (111) Йошкар-Ола-Ревда/Екатеринбург" от 11.04.2024, 2 465 руб.']
['13.02.2024 15:51', 'Аренда локомотива', '5. Продажа услуг аренды локомотивов', 'Завершен неудачно', 5087, 'ЦПУ КАЗАНЬ', nan, nan, nan, nan, '10. Холодный звонок', '0001-038494', 'Интерес "Аренда локомотива" от 13.02.2024']
['23.04.2024 10:59', 'Привлечение клиента на новое направление Йошкар-Ола - Балаково (код 041)', '6. Продажа услуг действующим клиентам', 'Завершен успешно', 5087, 'ЦПУ КАЗАНЬ', 591906.0, 100.0, nan, nan, '10. Холодный звонок', '0001-105828', 'Интерес "Привлечение клиента на новое направление Йошкар-Ола - Балаково (код 041)" от 23.04.2024, 591 906 руб.']
[4495, 'Нижегоро

In [252]:
               #Дополнительная информация которая тебе может помочь: {personal_dop_info}

# Класс для сервера

In [63]:
# Вводим ID Получаем  -> list ["ID", "%", "Вывод", "Рекомендации"]

class ML_AGEENT:
    def __init__(self) -> None:
        self.dataframe_interesting = pd.read_excel("Интересы.xls")
        self.dataframe_waitings = pd.read_excel("Обращения.xls")
        self.dataframe_transporting = pd.read_excel("Сортированный_Объем Перевозок.xlsx")
        self.dataframe_Vladimyr = pd.read_excel("МС_Владимирская область.xls")
        self.dataframe_Kirov = pd.read_excel("МС_Кировская область.xls")
        self.dataframe_NNov = pd.read_excel("МС_Нижегородская область.xls")
        self.dataframe_MaryEl = pd.read_excel("МС_Республика Марий Эл.xls")
        self.dataframe_Mordoviya = pd.read_excel("МС_Республика Мордовия.xls")
        self.dataframe_Tatarstan = pd.read_excel("МС_Республика Татарстан.xls")
        self.dataframe_Ydmurt = pd.read_excel("МС_Республика Удмуртия.xls")
        self.dataframe_Chyvash = pd.read_excel("МС_Республика Чувашия.xls")
        self.oblast_list = [
                            self.dataframe_Vladimyr, 
                            self.dataframe_Kirov, 
                            self.dataframe_NNov, 
                            self.dataframe_MaryEl, 
                            self.dataframe_Mordoviya, 
                            self.dataframe_Tatarstan,
                            self.dataframe_Ydmurt,
                            self.dataframe_Chyvash
                            ]


    def research(self, id:str) -> list:
        
        _, personal_data = self.research_oblasts(id)
        _, personal_seasons = self.seasons(id)
        _, personal_dop_info, sum = self.research_directions(id)
        persnal_problems = self.research_obrash(id)
        persnal_inters = self.research_interesting(id)
        
        
        
        
        prompt = f"""У нас есть информация о клиенте железнодорожной компании и ключевое значение для анализа:

Ключевое значение: {sum} — это общая сумма платежей клиента компании за последние 9 месяцев. Важно отметить, что:

   Если {sum} > 0, это значит, что клиент активен и продолжает делать заказы. В этом случае проанализируйте представленные данные и оцените вероятность ухода клиента, если вероятность составляет более 45%.
   Если {sum} = 0, клиент не осуществлял заказов в течение последних 9 месяцев, что означает высокий риск его ухода. В этом случае вероятность ухода составляет 100%, и нужно предоставить рекомендации для возвращения клиента.

Данные о клиенте:

  Информация о юридическом лице клиента, платежеспособности и рисках: {personal_data}
  Данные о грузоперевозках клиента за последние два года: {personal_seasons}
  Информация о взаимодействиях клиента с компанией, таких как обращения за консультированием, поддержка, жалобы, благодарности и т.д.: {persnal_inters}
  Данные о проблемах клиента, возникших ранее: {persnal_problems}

Ответ формируйте строго по следующему шаблону:

Вероятность ухода: <оценка вероятности ухода в процентах>
Обоснование: <обоснование выставленной вероятности и анализ, обоснованные на данных, рекомендации по удержанию>


Пожалуйста, используйте этот формат для ответа, чтобы он мог быть автоматически распознан системой. Также, на основе полученных данных, если {sum} > 0, оцените вероятность ухода клиента и предоставьте ИНДИВИДУАЛЬНЫЕ РЕКОМЕНДАЦИИ ДЛЯ УДЕРЖАНИЯ КЛИЕНТА, если вероятность ухода составляет более 45%. Если {sum} == 0, предложите действия для возвращения клиента с высокой вероятностью ухода (100%)

САМО ОБОСНОВАНИЕ ПРОСТАВЛЕННОЙ ВЕРОЯТНОСТИ ДОЛЖНО БЫТЬ КОНСТРУКТИВНЫМ И ЛОГИЧНЫМ
"""
        
        req=requests.get('http://127.0.0.1:5500',{'text': prompt})

            
        pattern = r"Вероятность ухода:\s*(\d+%)\nОбоснование:\s*(.*)"
        
        match = re.search(pattern, req.text, re.DOTALL)

        if match:
                # Извлечение данных из групп регулярного выражения
                churn_probability = match.group(1)  # Вероятность ухода
                justification = match.group(2)  # Обоснование
                
                # Формируем структурированный ответ
                structured_data_server = {
                    "churn_probability": churn_probability,
                    "justification_server": justification
                }
                
                print(structured_data_server)
                
                return structured_data_server
            
        else:
            
            print(req.text)
        
        # messages.append(HumanMessage(content=prompt))
        # res = chat(messages)
        # messages.append(res)
            
    
    
    def research_oblasts(self, id:str) -> str:
        for i in self.oblast_list:
            filtred = i[i["ID"] == id][["ID",
                                        "Размер компании.Наименование", 
                                        "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание", 
                                        "Карточка клиента (внешний источник).Индекс финансового риска Описание", 
                                        "Госконтракты.Тип контракта"]]
            if len(filtred.values.tolist()) >=1:
                break
            else:
                pass
        
        all_data = filtred.values.T.tolist()
        # print(all_data[0], all_data[1])
        all_data = [list(set(sublist)) for sublist in all_data]
        # print(all_data[0], all_data[1])
        
        final = "Этот клиент - " +   str(all_data[1]) + " C " + str(all_data[2]) + " С индексом финального риска: " + str(all_data[3]) + " И в гос контрактах он является: " + str(all_data[4])
        # print(final)
        
        return filtred, final
    
    def seasons(self, id:str) ->  str:
        
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        month_normal = years[0::2]
        month_in_numbers =  [[sum(subarray)] for subarray in month]
        month_in_tons = month_in_numbers[1::2]
        month_in_money = month_in_numbers[0::2]
        text = ""
        for i in range(len(month_normal)):
            text += f"Клиент в год, месяц - {month_normal[i]} перевёз {month_in_tons[i]} тон груза  на сумму: {month_in_money[i]}\n"
            
        # print(text)
        
        return filtred, text

    def research_directions(self, id:str):
        
        filtred=self.dataframe_transporting[self.dataframe_transporting['ID']==id]
        all_data = filtred.values.T.tolist()
        
        text=""
        sum=0
        
        for i in range(len(all_data[3])):

            text+="| Этот клиент везет из - " + all_data[1][i] + " в - " + all_data[2][i] + "| Код товара - " + str(all_data[3][i])+"| Категория груза - "+all_data[4][i]+"|\n"
            
        # print (text)
        
        
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        
        
        for i in month[-8::2]:
            
            for j in i:
                
                sum+=j
                
        
        
        print(years,month)
        
        
        return all_data, text,sum

    def research_interesting(self, id:str) -> str:
        filtred = self.dataframe_interesting[self.dataframe_interesting["ID"] == id][["Состояние", "Канал первичного интереса"]]

        all_data = filtred.values.T.tolist()
        all_data = [list(set(sublist)) for sublist in all_data]

        string = ""
        ls = []

        for i in filtred.values:
            string += str(i[0]) + " " + str(i[1]) + ", "

        for i in all_data[0]:
            for j in all_data[1]:
                if string.count(f"{i} {j}") != 0:
                    ls.append([str(i) + " " + str(j), string.count(f"{i} {j}")])

        for i in ls:
            string += i[0] + " " + str(i[1]) + "\n"

        return string
    

    def research_obrash(self, id:str) -> str:
        filtred = self.dataframe_waitings[self.dataframe_waitings["ID"] == id][["Тема вопроса"]]

        all_data = filtred.values.T.tolist()
        try:
            
            string = ""
            for i in all_data:
                string += i[0] + "\n"

            print(string)
        except:
            string = "Обращений не обнаружено"
        
        return string
 

# Класс для обычной обработки

In [66]:
# Вводим ID Получаем  -> list ["ID", "%", "Вывод", "Рекомендации"]

class ML_AGEENT:
    def __init__(self) -> None:
        self.dataframe_interesting = pd.read_excel("Интересы.xls")
        self.dataframe_waitings = pd.read_excel("Обращения.xls")
        self.dataframe_transporting = pd.read_excel("Сортированный_Объем Перевозок.xlsx")
        self.dataframe_Vladimyr = pd.read_excel("МС_Владимирская область.xls")
        self.dataframe_Kirov = pd.read_excel("МС_Кировская область.xls")
        self.dataframe_NNov = pd.read_excel("МС_Нижегородская область.xls")
        self.dataframe_MaryEl = pd.read_excel("МС_Республика Марий Эл.xls")
        self.dataframe_Mordoviya = pd.read_excel("МС_Республика Мордовия.xls")
        self.dataframe_Tatarstan = pd.read_excel("МС_Республика Татарстан.xls")
        self.dataframe_Ydmurt = pd.read_excel("МС_Республика Удмуртия.xls")
        self.dataframe_Chyvash = pd.read_excel("МС_Республика Чувашия.xls")
        self.oblast_list = [
                            self.dataframe_Vladimyr, 
                            self.dataframe_Kirov, 
                            self.dataframe_NNov, 
                            self.dataframe_MaryEl, 
                            self.dataframe_Mordoviya, 
                            self.dataframe_Tatarstan,
                            self.dataframe_Ydmurt,
                            self.dataframe_Chyvash
                            ]


    def research(self, id:str) -> list:
        
        _, personal_data = self.research_oblasts(id)
        _, personal_seasons = self.seasons(id)
        _, personal_dop_info, sum = self.research_directions(id)
        persnal_problems = self.research_obrash(id)
        persnal_inters = self.research_interesting(id)
        
        
        
        
        prompt = f"""У нас есть информация о клиенте железнодорожной компании и ключевое значение для анализа:

Ключевое значение: {sum} — это общая сумма платежей клиента компании за последние 9 месяцев. Важно отметить, что:

   Если {sum} > 0, это значит, что клиент активен и продолжает делать заказы. В этом случае проанализируйте представленные данные и оцените вероятность ухода клиента, если вероятность составляет более 45%.
   Если {sum} = 0, клиент не осуществлял заказов в течение последних 9 месяцев, что означает высокий риск его ухода. В этом случае вероятность ухода составляет 100%, и нужно предоставить рекомендации для возвращения клиента.

Данные о клиенте:

  Информация о юридическом лице клиента, платежеспособности и рисках: {personal_data}
  Данные о грузоперевозках клиента за последние два года: {personal_seasons}
  Информация о взаимодействиях клиента с компанией, таких как обращения за консультированием, поддержка, жалобы, благодарности и т.д.: {persnal_inters}
  Данные о проблемах клиента, возникших ранее: {persnal_problems}

Ответ формируйте строго по следующему шаблону:

Вероятность ухода: <оценка вероятности ухода в процентах>
Обоснование: <обоснование выставленной вероятности и анализ, обоснованные на данных, рекомендации по удержанию>


Пожалуйста, используйте этот формат для ответа, чтобы он мог быть автоматически распознан системой. Также, на основе полученных данных, если {sum} > 0, оцените вероятность ухода клиента и предоставьте ИНДИВИДУАЛЬНЫЕ РЕКОМЕНДАЦИИ ДЛЯ УДЕРЖАНИЯ КЛИЕНТА, если вероятность ухода составляет более 45%. Если {sum} == 0, предложите действия для возвращения клиента с высокой вероятностью ухода (100%)

САМО ОБОСНОВАНИЕ ПРОСТАВЛЕННОЙ ВЕРОЯТНОСТИ ДОЛЖНО БЫТЬ КОНСТРУКТИВНЫМ И ЛОГИЧНЫМ
"""
        
        messages.append(HumanMessage(content=prompt))
        res = chat(messages)
        messages.append(res)
        
        pattern = r"Вероятность ухода:\s*(\d+%)\nОбоснование:\s*(.*)"
        
        match = re.search(pattern, res.content, re.DOTALL)

        if match:
            # Извлечение данных из групп регулярного выражения
            churn_probability = match.group(1)  # Вероятность ухода
            justification = match.group(2)  # Обоснование
            
            # Формируем структурированный ответ
            structured_data = {
                "churn_probability": churn_probability,
                "justification_base": justification
            }
            _
            return structured_data
    
    
    def research_oblasts(self, id:str) -> str:
        for i in self.oblast_list:
            filtred = i[i["ID"] == id][["ID",
                                        "Размер компании.Наименование", 
                                        "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание", 
                                        "Карточка клиента (внешний источник).Индекс финансового риска Описание", 
                                        "Госконтракты.Тип контракта"]]
            if len(filtred.values.tolist()) >=1:
                break
            else:
                pass
        
        all_data = filtred.values.T.tolist()
        # print(all_data[0], all_data[1])
        all_data = [list(set(sublist)) for sublist in all_data]
        # print(all_data[0], all_data[1])
        
        final = "Этот клиент - " +   str(all_data[1]) + " C " + str(all_data[2]) + " С индексом финального риска: " + str(all_data[3]) + " И в гос контрактах он является: " + str(all_data[4])
        # print(final)
        
        return filtred, final
    
    def seasons(self, id:str) ->  str:
        
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        month_normal = years[0::2]
        month_in_numbers =  [[sum(subarray)] for subarray in month]
        month_in_tons = month_in_numbers[1::2]
        month_in_money = month_in_numbers[0::2]
        text = ""
        for i in range(len(month_normal)):
            text += f"Клиент в год, месяц - {month_normal[i]} перевёз {month_in_tons[i]} тон груза  на сумму: {month_in_money[i]}\n"
            
        # print(text)
        
        return filtred, text

    def research_directions(self, id:str):
        
        filtred=self.dataframe_transporting[self.dataframe_transporting['ID']==id]
        all_data = filtred.values.T.tolist()
        
        text=""
        sum=0
        
        for i in range(len(all_data[3])):

            text+="| Этот клиент везет из - " + all_data[1][i] + " в - " + all_data[2][i] + "| Код товара - " + str(all_data[3][i])+"| Категория груза - "+all_data[4][i]+"|\n"
            
        # print (text)
        
        
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        
        
        for i in month[-8::2]:
            
            for j in i:
                
                sum+=j
                
        
        
        print(years,month)
        
        
        return all_data, text,sum

    def research_interesting(self, id:str) -> str:
        filtred = self.dataframe_interesting[self.dataframe_interesting["ID"] == id][["Состояние", "Канал первичного интереса"]]

        all_data = filtred.values.T.tolist()
        all_data = [list(set(sublist)) for sublist in all_data]

        string = ""
        ls = []

        for i in filtred.values:
            string += str(i[0]) + " " + str(i[1]) + ", "

        for i in all_data[0]:
            for j in all_data[1]:
                if string.count(f"{i} {j}") != 0:
                    ls.append([str(i) + " " + str(j), string.count(f"{i} {j}")])

        for i in ls:
            string += i[0] + " " + str(i[1]) + "\n"

        return string
    

    def research_obrash(self, id:str) -> str:
        filtred = self.dataframe_waitings[self.dataframe_waitings["ID"] == id][["Тема вопроса"]]

        all_data = filtred.values.T.tolist()
        try:
            
            string = ""
            for i in all_data:
                string += i[0] + "\n"

            print(string)
        except:
            string = "Обращений не обнаружено"
        
        return string

In [67]:
agent = ML_AGEENT()

In [68]:
agent.research(id)

['2022/01', '2022/01.1', '2022/02', '2022/02.1', '2022/03', '2022/03.1', '2022/04', '2022/04.1', '2022/05', '2022/05.1', '2022/06', '2022/06.1', '2022/07', '2022/07.1', '2022/08', '2022/08.1', '2022/09', '2022/09.1', '2022/10', '2022/10.1', '2022/11', '2022/11.1', '2022/12', '2022/12.1', '2023/01', '2023/01.1', '2023/02', '2023/02.1', '2023/03', '2023/03.1', '2023/04', '2023/04.1', '2023/05', '2023/05.1', '2023/06', '2023/06.1', '2023/07', '2023/07.1', '2023/08', '2023/08.1', '2023/09', '2023/09.1', '2023/10', '2023/10.1', '2023/11', '2023/11.1', '2023/12', '2023/12.1', '2024/01', '2024/01.1', '2024/02', '2024/02.1', '2024/03', '2024/03.1', '2024/04', '2024/04.1', '2024/05', '2024/05.1', '2024/06', '2024/06.1', '2024/07', '2024/07.1', '2024/08', '2024/08.1'] [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 320452, 1169796, 697781, 0, 1344006], [0, 0, 310, 809, 707, 0, 933], [0, 0, 767241, 3112059, 947725, 975279, 0], [0, 0, 736, 2156, 952, 553, 0], [0, 669897, 0, 89865, 1167449, 0

{'churn_probability': '25%',
 'justification_base': 'Анализ данных показывает, что клиент активно использует услуги компании, о чем свидетельствует положительное значение общей суммы платежей за последние 9 месяцев (14803874). Однако, несмотря на активность, общая сумма платежей снижается с течением времени, что может указывать на возможное снижение интереса к услугам компании. Рекомендуется провести углубленный анализ поведения клиента и выявить причины снижения активности, а также разработать стратегию удержания, включающую улучшение качества услуг и персонализированные предложения.'}

In [61]:
req=requests.get('http://127.0.0.1:5500',{'text': "How are you?"})

if req.status_code==200:
            
    print(req.text)
            
else:
            
    print(req.text)

As an artificial intelligence, I don't have feelings, but I'm functioning as expected. Thank you for asking! How can I assist you today?
